<a href="https://colab.research.google.com/github/dderaad/Hackathon-Summer-2021/blob/main/clfs_mk1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
# All of the paths for the relevant data sets are compiled into a list of urls
 
PATH = "https://raw.githubusercontent.com/dderaad/Hackathon-Summer-2021/main/"
files = [
         "prediction/prediction.csv", 
         "test_data/test_LABELS.csv", 
         "test_data/test_feature_names.csv", 
         "test_data/test_expression.csv.gz", 
         "train_data/train_labels.csv", 
         "train_data/train_feature_names.csv", 
         "train_data/train_expression.csv.gz"
         ]
urls = [PATH + x for x in files]
urls

['https://raw.githubusercontent.com/dderaad/Hackathon-Summer-2021/main/prediction/prediction.csv',
 'https://raw.githubusercontent.com/dderaad/Hackathon-Summer-2021/main/test_data/test_LABELS.csv',
 'https://raw.githubusercontent.com/dderaad/Hackathon-Summer-2021/main/test_data/test_feature_names.csv',
 'https://raw.githubusercontent.com/dderaad/Hackathon-Summer-2021/main/test_data/test_expression.csv.gz',
 'https://raw.githubusercontent.com/dderaad/Hackathon-Summer-2021/main/train_data/train_labels.csv',
 'https://raw.githubusercontent.com/dderaad/Hackathon-Summer-2021/main/train_data/train_feature_names.csv',
 'https://raw.githubusercontent.com/dderaad/Hackathon-Summer-2021/main/train_data/train_expression.csv.gz']

In [3]:
# The datasets are read into a python dictionary with their simplified path names as their keys
 
data_sets = {}
for url, file_name in zip(urls, files):
  data_sets[file_name] = pd.read_csv(url)
 
for data_set_name in data_sets:
  print(data_set_name, "\n", data_sets[data_set_name].head(5), "\n")

prediction/prediction.csv 
          age sample_id
0  57.144041  12fa2c69
1  57.144041  4d41ce5f
2  57.144041  82f6b2bb
3  57.144041  81161a04
4  57.144041  4370df33 

test_data/test_LABELS.csv 
   sample_id group
0  12fa2c69     C
1  4d41ce5f     A
2  82f6b2bb     C
3  81161a04     A
4  4370df33     B 

test_data/test_feature_names.csv 
               gene_id    symbol
0  ENSG00000000003.14    TSPAN6
1   ENSG00000000005.5      TNMD
2  ENSG00000000419.12      DPM1
3  ENSG00000000457.13     SCYL3
4  ENSG00000000460.16  C1orf112 

test_data/test_expression.csv.gz 
    ENSG00000000003.14  ENSG00000000005.5  ...  ENSG00000283698.1  ENSG00000283699.1
0              216130               3724  ...                  1                  1
1              196089                671  ...                 48                  0
2              455669             123617  ...                  1                  0
3               92006                400  ...                 28                  1
4         

In [4]:
labelled_training_data = pd.concat([data_sets["train_data/train_labels.csv"],
                                    data_sets["train_data/train_expression.csv.gz"] 
                                    ], axis=1)
labelled_training_data.head(10)

,sample_id,age,group,ENSG00000000003.14,ENSG00000000005.5,ENSG00000000419.12,ENSG00000000457.13,ENSG00000000460.16,ENSG00000000938.12,ENSG00000000971.15,ENSG00000001036.13,ENSG00000001084.10,ENSG00000001167.14,ENSG00000001460.17,ENSG00000001461.16,ENSG00000001561.6,ENSG00000001617.11,ENSG00000001626.14,ENSG00000001629.9,ENSG00000001630.15,ENSG00000002016.17,ENSG00000002079.12,ENSG00000002549.12,ENSG00000002586.18,ENSG00000002587.9,ENSG00000002726.20,ENSG00000002745.12,ENSG00000002746.14,ENSG00000002834.17,ENSG00000002919.14,ENSG00000002933.7,ENSG00000003096.13,ENSG00000003137.8,ENSG00000003147.17,ENSG00000003249.13,ENSG00000003400.14,ENSG00000003402.19,ENSG00000003436.14,ENSG00000003756.16,ENSG00000003987.13,...,ENSG00000283636.1,ENSG00000283638.1,ENSG00000283639.1,ENSG00000283644.1,ENSG00000283645.1,ENSG00000283646.1,ENSG00000283647.1,ENSG00000283648.1,ENSG00000283652.1,ENSG00000283653.1,ENSG00000283654.1,ENSG00000283656.1,ENSG00000283657.1,ENSG00000283659.1,ENSG00000283662.1,ENSG00000283663.1,ENSG00000283667.1,ENSG00000283669.1,ENSG00000283672.1,ENSG00000283673.1,ENSG00000283674.1,ENSG00000283676.1,ENSG00000283678.1,ENSG00000283679.1,ENSG00000283680.1,ENSG00000283682.1,ENSG00000283683.1,ENSG00000283684.1,ENSG00000283686.1,ENSG00000283689.1,ENSG00000283690.1,ENSG00000283691.1,ENSG00000283692.1,ENSG00000283693.1,ENSG00000283694.1,ENSG00000283695.1,ENSG00000283696.1,ENSG00000283697.1,ENSG00000283698.1,ENSG00000283699.1
0,e1b41edc,55,A,62713,192,177091,135168,73181,23403,95254,165911,180606,368118,106610,320565,446433,702977,2844,418633,456407,84908,2371,319489,265113,12173,862,384,21100,1774586,152600,39672,15337,3921,370822,771708,27232,466489,44714,696054,193179,...,1,858,0,0,2,5323,0,1,35486,0,2799,0,241,49,10462,854,11023,0,36,0,17648,110,1,0,1,1,144,1335,2,5602,1,644,0,0,40,0,2886,8265,1,1
1,234e8e31,62,A,124182,4870,121707,64541,49122,34556,104741,96063,74906,75056,16688,42124,12696,376101,33,151094,185958,12696,336,161551,480428,10300,2930,143,683,2520161,210741,575543,1419,21499,122818,332751,21867,179530,25954,284438,5256,...,0,0,0,1,1,95,0,1,26394,0,431,1,0,1,1062,0,3805,1,1,49,2559,107,3,0,0,0,68,722,3,912,0,121,0,0,0,69,335,3797,46,1
2,df005c89,52,A,285105,385,132324,201131,112838,36724,283646,209734,160594,262819,47134,123372,74044,813642,1717,270598,567574,31926,577,397680,1272208,24440,4049,568,1382,7125969,78444,145272,39394,27243,288834,227256,39535,362362,67972,633312,8087,...,1,0,0,33,1,2260,1,1,33308,0,2032,54,67,1,1206,1194,1997,1,0,0,1881,208,2,0,0,1,1,4441,0,2334,20,163,1,1,66,47,3801,13738,48,1
3,818437b2,42,A,277543,480,87439,113100,67013,34519,186042,146815,88497,150401,37532,153206,58714,377449,1751,175167,198559,22777,454,321259,953790,12415,1564,376,1881,1509005,115912,167189,13526,6493,183236,73275,33395,287364,52828,398867,3810,...,1,0,0,0,0,150819,2,96,48200,0,2126,19,0,0,577,1565,746,0,0,0,2713,738,2,0,0,0,192,2038,0,2832,1,253,1,0,1,0,3514,7445,48,0
4,8b7f7e4d,63,A,89628,288,89571,128793,59066,22495,153702,139875,124523,73579,40103,75714,25724,162708,443,100607,78755,18375,125,672231,768209,6387,1316,193,433,908969,108943,174869,11493,8066,91760,30703,33944,280800,38244,359759,3718,...,0,96,0,0,0,2318,0,231,51200,2,1432,0,2,0,192,701,504,1,41,1,1913,59,0,1,48,0,52,1478,1,825,0,147,0,0,17,0,3516,6047,1,0
5,65846c93,77,A,361848,101,230960,224878,160255,67476,444287,741162,343287,238818,44584,170431,211562,741479,748,619979,987440,35049,1269,508909,2561747,44109,376985,253,8164,37416751,89608,249826,29593,56281,399831,493403,38547,500612,93731,454365,22194,...,1,0,1,14,1,5384,1,0,53927,1,3399,56,46,1,1901,1512,968,96,0,0,8915,331,9,0,350,0,274,6114,1,4179,0,100,0,50,1,0,3309,4211,399,0
6,3441dfca,50,A,84670,100,546749,282111,169075,40453,454057,636794,173055,268553,52323,229088,292262,450180,1317,558099,575118,37068,495,584224,1348411,16008,5381,201,2135,1926755,136915,198336,15854,27179,542427,390870,46609,602239,68637,494831,7726,...,0,20,0,0,2,478,1,47,125505,0,1834,1,118,0,153,279,2081,0,1,1,2

In [5]:
labelled_testing_data = pd.concat([data_sets["test_data/test_LABELS.csv"],
                                    data_sets["test_data/test_expression.csv.gz"] 
                                    ], axis=1)
labelled_testing_data.head(10)

,sample_id,group,ENSG00000000003.14,ENSG00000000005.5,ENSG00000000419.12,ENSG00000000457.13,ENSG00000000460.16,ENSG00000000938.12,ENSG00000000971.15,ENSG00000001036.13,ENSG00000001084.10,ENSG00000001167.14,ENSG00000001460.17,ENSG00000001461.16,ENSG00000001561.6,ENSG00000001617.11,ENSG00000001626.14,ENSG00000001629.9,ENSG00000001630.15,ENSG00000002016.17,ENSG00000002079.12,ENSG00000002549.12,ENSG00000002586.18,ENSG00000002587.9,ENSG00000002726.20,ENSG00000002745.12,ENSG00000002746.14,ENSG00000002834.17,ENSG00000002919.14,ENSG00000002933.7,ENSG00000003096.13,ENSG00000003137.8,ENSG00000003147.17,ENSG00000003249.13,ENSG00000003400.14,ENSG00000003402.19,ENSG00000003436.14,ENSG00000003756.16,ENSG00000003987.13,ENSG00000003989.16,...,ENSG00000283636.1,ENSG00000283638.1,ENSG00000283639.1,ENSG00000283644.1,ENSG00000283645.1,ENSG00000283646.1,ENSG00000283647.1,ENSG00000283648.1,ENSG00000283652.1,ENSG00000283653.1,ENSG00000283654.1,ENSG00000283656.1,ENSG00000283657.1,ENSG00000283659.1,ENSG00000283662.1,ENSG00000283663.1,ENSG00000283667.1,ENSG00000283669.1,ENSG00000283672.1,ENSG00000283673.1,ENSG00000283674.1,ENSG00000283676.1,ENSG00000283678.1,ENSG00000283679.1,ENSG00000283680.1,ENSG00000283682.1,ENSG00000283683.1,ENSG00000283684.1,ENSG00000283686.1,ENSG00000283689.1,ENSG00000283690.1,ENSG00000283691.1,ENSG00000283692.1,ENSG00000283693.1,ENSG00000283694.1,ENSG00000283695.1,ENSG00000283696.1,ENSG00000283697.1,ENSG00000283698.1,ENSG00000283699.1
0,12fa2c69,C,216130,3724,158501,169431,72136,239413,1958773,405054,190105,111787,31063,132119,114077,294983,18470,258495,148914,69474,786,503892,2358522,35566,500,684,2020,749151,110511,365163,30185,80184,234757,45482,128455,1512878,564145,1401215,21474,710939,...,1,30,49,0,0,47729,230,0,37719,0,988,583,1514,1,0,12020,2525,0,35,1,49296,182,1,0,1,19,2636,8473,0,4080,1,160,0,151,23,2,2382,11417,1,1
1,4d41ce5f,A,196089,671,96434,112703,67922,15916,209234,105360,192348,110960,41122,150374,51857,614533,512,273371,313928,34375,878,264207,931650,7122,763,430,1104,4700454,56400,99863,15356,11567,257478,72267,24287,291029,49599,476567,2662,291616,...,63,144,0,48,1,5936,127,0,28180,2,1057,350,148,0,314,1419,1290,88,0,0,933,110,23,0,0,1,182,2365,2,6720,0,186,0,1,0,0,1895,10016,48,0
2,82f6b2bb,C,455669,123617,217274,174936,88070,382592,817841,396940,397913,198589,73080,163311,215405,785858,9219,317722,122683,117439,471,758389,3025847,67874,1064,725,5031,1786045,187573,296825,20885,437483,221538,6445,163422,3317326,540599,1957008,20193,541206,...,149,1,1,129,0,7153,219,37,41025,61,3662,1,570,1,1,579,2529,0,0,0,4419,0,0,26,0,1,10781,4055,0,1508,0,71,1,0,0,1,2710,9278,1,0
3,81161a04,A,92006,400,271295,444488,242673,50198,359611,310289,783368,753853,105544,498159,581568,865985,750,699658,916791,88424,1048,957894,1046496,44048,695,265,3779,3697981,248832,115369,10474,10192,677240,272138,91950,972449,97360,1296196,8877,5546393,...,51,198,0,0,0,22805,0,1,51889,0,5224,348,430,100,1539,99,3198,0,1,0,3797,677,5,0,803,1,169,3070,1,2003,1,544,1,1,59,52,6067,27156,28,1
4,4370df33,B,311516,108041,143606,134343,70622,261239,1046396,238826,187343,172348,30365,77754,223768,1220282,2142,282388,189056,62412,495,767745,2095274,20109,683,487,3525,974222,115994,260399,15503,287725,236076,8490,203558,1769920,915327,1330813,19770,747324,...,0,28,0,0,1,4033,152,0,39738,0,1255,0,192,77,1,904,1143,2,0,151,3637,0,0,0,0,0,493,11201,1,697,0,238,152,0,0,52,5699,5548,76,1
5,0663bdbc,A,71028,1396,193283,185896,109605,69556,196118,330042,219307,280310,69168,488790,171483,529157,514,425181,285540,16484,889,965354,582746,53328,260316,381,2890,3058218,151071,791050,60694,7160,601284,320149,141483,791427,36465,427639,91264,7510602,...,1,1,0,52,2,424,4,262,184957,2,2010,85,65,0,1216,9084,1988,0,1,0,3716,8,2,1,0,1,100,1260,28,3653,1,377,0,0,152,105,3272,46137,52,0
6,1e34b941,A,17383,383,133048,176637,105399,39615,32340,370802,174045,134592,50765,122883,84726,351294,22,130993,202692,29828,468,320833,1506792,22914,1340,523,89,1832327,109756,965685,220

In [6]:
#labelled_testing_data.describe()

,ENSG00000000003.14,ENSG00000000005.5,ENSG00000000419.12,ENSG00000000457.13,ENSG00000000460.16,ENSG00000000938.12,ENSG00000000971.15,ENSG00000001036.13,ENSG00000001084.10,ENSG00000001167.14,ENSG00000001460.17,ENSG00000001461.16,ENSG00000001561.6,ENSG00000001617.11,ENSG00000001626.14,ENSG00000001629.9,ENSG00000001630.15,ENSG00000002016.17,ENSG00000002079.12,ENSG00000002549.12,ENSG00000002586.18,ENSG00000002587.9,ENSG00000002726.20,ENSG00000002745.12,ENSG00000002746.14,ENSG00000002834.17,ENSG00000002919.14,ENSG00000002933.7,ENSG00000003096.13,ENSG00000003137.8,ENSG00000003147.17,ENSG00000003249.13,ENSG00000003400.14,ENSG00000003402.19,ENSG00000003436.14,ENSG00000003756.16,ENSG00000003987.13,ENSG00000003989.16,ENSG00000004059.10,ENSG00000004139.13,...,ENSG00000283636.1,ENSG00000283638.1,ENSG00000283639.1,ENSG00000283644.1,ENSG00000283645.1,ENSG00000283646.1,ENSG00000283647.1,ENSG00000283648.1,ENSG00000283652.1,ENSG00000283653.1,ENSG00000283654.1,ENSG00000283656.1,ENSG00000283657.1,ENSG00000283659.1,ENSG00000283662.1,ENSG00000283663.1,ENSG00000283667.1,ENSG00000283669.1,ENSG00000283672.1,ENSG00000283673.1,ENSG00000283674.1,ENSG00000283676.1,ENSG00000283678.1,ENSG00000283679.1,ENSG00000283680.1,ENSG00000283682.1,ENSG00000283683.1,ENSG00000283684.1,ENSG00000283686.1,ENSG00000283689.1,ENSG00000283690.1,ENSG00000283691.1,ENSG00000283692.1,ENSG00000283693.1,ENSG00000283694.1,ENSG00000283695.1,ENSG00000283696.1,ENSG00000283697.1,ENSG00000283698.1,ENSG00000283699.1
count,9.650000e+02,9.650000e+02,9.650000e+02,9.650000e+02,965.000000,9.650000e+02,9.650000e+02,9.650000e+02,9.650000e+02,9.650000e+02,965.000000,9.650000e+02,9.650000e+02,9.650000e+02,965.000000,9.650000e+02,9.650000e+02,965.000000,965.000000,9.650000e+02,9.650000e+02,965.000000,9.650000e+02,965.000000,965.000000,9.650000e+02,9.650000e+02,9.650000e+02,965.000000,9.650000e+02,9.650000e+02,9.650000e+02,9.650000e+02,9.650000e+02,9.650000e+02,9.650000e+02,965.000000,9.650000e+02,9.650000e+02,9.650000e+02,...,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,9.650000e+02,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000
mean,3.109436e+05,2.224690e+04,2.249264e+05,2.419759e+05,149125.987565,9.266597e+04,5.451912e+05,3.805445e+05,2.710049e+05,3.279705e+05,69525.163731,2.609119e+05,1.950115e+05,5.932903e+05,6511.676684,4.505066e+05,4.338533e+05,59780.418653,1191.692228,7.250090e+05,1.232703e+06,30635.345078,1.993086e+04,1034.469430,4089.312953,2.376375e+06,1.662944e+05,2.602264e+05,50768.617617,1.007118e+05,3.223093e+05,2.133747e+05,8.674132e+04,8.679118e+05,2.214679e+05,7.929395e+05,18156.200000,1.437104e+06,8.649572e+05,2.814672e+05,...,42.064249,108.383420,2.158549,37.283938,1.528497,11181.894301,104.198964,79.615544,7.067158e+04,6.009326,2464.474611,94.803109,416.243523,9.948187,1376.447668,2655.878756,2460.957513,16.579275,27.479793,8.366839,6370.291192,144.734715,3.340933,8.240415,198.907772,17.102591,3233.406218,3677.686010,7.900518,3242.048705,9.889119,244.938860,13.778238,8.593782,21.310881,15.988601,4304.810363,11272.650777,34.871503,5.164767
std,2.624927e+05,7.556629e+04,1.506738e+05,1.401705e+05,96949.609637,1.208877e+05,8.475259e+05,2.257942e+05,2.012401e+05,2.574910e+05,35035.530867,2.153647e+05,2.458923e+05,4.066961e+05,33175.269349,2.729997e+05,3.061142e+05,40563.817414,5028.931156,5.000028e+05,7.484085e+05,24785.085163,1.306710e+05,6028.735818,5196.087016,2.709163e+06,1.242722e+05,2.883022e+05,83105.096904,2.347681e+05,1.999984e+05,2.229132e+05,9.126413e+04,7.609621e+05,3.293674e+05,5.908230e+05,22134.021532,2.585567e+06,4.440932e+05,2.124105e+05,...,112.559758,279.736394,9.351190,87.125584,6.687331,39496.076126,265.261645,272.007754,9.751241e+0

In [7]:
#labelled_training_data.describe()

,age,ENSG00000000003.14,ENSG00000000005.5,ENSG00000000419.12,ENSG00000000457.13,ENSG00000000460.16,ENSG00000000938.12,ENSG00000000971.15,ENSG00000001036.13,ENSG00000001084.10,ENSG00000001167.14,ENSG00000001460.17,ENSG00000001461.16,ENSG00000001561.6,ENSG00000001617.11,ENSG00000001626.14,ENSG00000001629.9,ENSG00000001630.15,ENSG00000002016.17,ENSG00000002079.12,ENSG00000002549.12,ENSG00000002586.18,ENSG00000002587.9,ENSG00000002726.20,ENSG00000002745.12,ENSG00000002746.14,ENSG00000002834.17,ENSG00000002919.14,ENSG00000002933.7,ENSG00000003096.13,ENSG00000003137.8,ENSG00000003147.17,ENSG00000003249.13,ENSG00000003400.14,ENSG00000003402.19,ENSG00000003436.14,ENSG00000003756.16,ENSG00000003987.13,ENSG00000003989.16,ENSG00000004059.10,...,ENSG00000283636.1,ENSG00000283638.1,ENSG00000283639.1,ENSG00000283644.1,ENSG00000283645.1,ENSG00000283646.1,ENSG00000283647.1,ENSG00000283648.1,ENSG00000283652.1,ENSG00000283653.1,ENSG00000283654.1,ENSG00000283656.1,ENSG00000283657.1,ENSG00000283659.1,ENSG00000283662.1,ENSG00000283663.1,ENSG00000283667.1,ENSG00000283669.1,ENSG00000283672.1,ENSG00000283673.1,ENSG00000283674.1,ENSG00000283676.1,ENSG00000283678.1,ENSG00000283679.1,ENSG00000283680.1,ENSG00000283682.1,ENSG00000283683.1,ENSG00000283684.1,ENSG00000283686.1,ENSG00000283689.1,ENSG00000283690.1,ENSG00000283691.1,ENSG00000283692.1,ENSG00000283693.1,ENSG00000283694.1,ENSG00000283695.1,ENSG00000283696.1,ENSG00000283697.1,ENSG00000283698.1,ENSG00000283699.1
count,965.000000,9.650000e+02,9.650000e+02,9.650000e+02,965.000000,9.650000e+02,9.650000e+02,9.650000e+02,9.650000e+02,9.650000e+02,9.650000e+02,965.000000,9.650000e+02,9.650000e+02,9.650000e+02,965.000000,9.650000e+02,9.650000e+02,965.000000,965.000000,9.650000e+02,9.650000e+02,965.000000,9.650000e+02,965.000000,965.000000,9.650000e+02,9.650000e+02,9.650000e+02,965.000000,9.650000e+02,9.650000e+02,9.650000e+02,9.650000e+02,9.650000e+02,9.650000e+02,9.650000e+02,965.000000,9.650000e+02,9.650000e+02,...,965.000000,965.00000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000
mean,54.144041,2.688726e+05,6.220241e+04,1.782797e+05,165383.613472,9.506898e+04,1.693975e+05,9.740888e+05,3.034849e+05,2.417480e+05,2.203811e+05,52968.795855,1.601159e+05,1.834135e+05,4.796825e+05,7217.476684,3.415741e+05,2.953447e+05,71074.817617,818.877720,6.256434e+05,1.542931e+06,28769.582383,9.361534e+03,1833.750259,4386.429016,1.657129e+06,1.338258e+05,2.598405e+05,39179.166839,2.387156e+05,2.279921e+05,1.121975e+05,1.241625e+05,1.341186e+06,3.847668e+05,1.082789e+06,18492.682902,1.066505e+06,6.424344e+05,...,36.722280,83.21658,2.611399,36.029016,1.238342,8359.201036,238.588601,48.105699,48214.951295,5.759585,1839.476684,62.088083,594.505699,9.262176,628.192746,3157.950259,2224.329534,26.579275,33.244560,8.982383,8043.293264,92.375130,3.567876,14.678756,30.682902,13.872539,7456.653886,6932.463212,24.084974,2173.545078,6.385492,185.630052,17.051813,8.078756,15.292228,16.936788,3336.880829,8290.696373,30.846632,4.958549
std,13.327155,2.017592e+05,1.143837e+05,1.383469e+05,118670.644537,8.498915e+04,1.934962e+05,1.215090e+06,2.379000e+05,1.672591e+05,2.661023e+05,31018.379301,1.718471e+05,1.740569e+05,3.682904e+05,20059.949085,2.434584e+05,4.424800e+05,51227.032330,1616.138019,4.831726e+05,9.285222e+05,22774.655375,8.233780e+04,7164.750911,6023.336670,2.443910e+06,1.016397e+05,2.378535e+05,54514.030196,3.103150e+05,1.738201e+05,1.905358e+05,1.239374e+05,1.203594e+06,3.709788e+05,7.798868e+05,17421.491404,1.653809e+06,3.876300e+05,...,79.816878,176.87124,13.062619,87.582310,5.206442,21588.047823,506.211377,232.017758,54194.261869,21.862837,1439.512122,193.3264

In [8]:
from sklearn.dummy import DummyClassifier
strategies = ["most_frequent", "uniform", "constant"]
dummy_classifiers = {f"Dummy Classifier ({i})": DummyClassifier(i, constant=np.median(labelled_training_data["age"])) for i in strategies}

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

clfs = {
    "Decision Tree (depth 5)": DecisionTreeClassifier(max_depth=5),
    "Random Forest (depth 5)": RandomForestClassifier(max_depth=5),
    #"AdaBoost": AdaBoostClassifier(),
    "Linear SVM": SVC(kernel="linear", C=0.025),
    "RBF SVM": SVC(gamma=2, C=1),
    #"Gaussian Process": GaussianProcessClassifier(1.0 * RBF(1.0))
}
 
classifiers = {}
classifiers.update(dummy_classifiers)
classifiers.update(clfs)

In [9]:
# format: X, y
Xy_data_set = [
               labelled_training_data.drop(["sample_id", "group", "age"], axis=1),
               labelled_training_data["age"]
               ]
Xy_data_set
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(Xy_data_set[0])
Xy_data_set_scaled = Xy_data_set
Xy_data_set_scaled[0] = scaler.transform(Xy_data_set[0])
 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Xy_data_set_scaled[0], Xy_data_set_scaled[1], test_size=0.15, random_state=1)
X_train

array([[-0.76579728, -0.44928642, -0.72413344, ..., -0.65322974,
         0.07820971, -0.29457974],
       [-0.48963698, -0.40206117, -0.46431108, ..., -0.40155247,
        -0.4377888 , -0.29457974],
       [ 1.67088383,  0.61983771, -0.26839749, ..., -0.55758705,
        -0.45296523, -0.23517129],
       ...,
       [-0.77398455, -0.47955117, -0.42146899, ..., -0.48987895,
        -0.2101424 , -0.17576283],
       [-0.0281747 , -0.54408681,  1.73939505, ...,  2.30105172,
        -0.46814166, -0.29457974],
       [ 0.44333446, -0.52646153, -0.23893451, ...,  0.28350987,
        -0.45296523, -0.23517129]])

In [10]:
from sklearn.metrics import mean_squared_error
 
for clf_name in classifiers:
  classifiers[clf_name].fit(X_train, y_train)
  score = classifiers[clf_name].score(X_test, y_test)
  y_pred = classifiers[clf_name].predict(X_test)
  print(f'{clf_name}\nMSE: {mean_squared_error(y_test, y_pred)} \n{score}\n')

Dummy Classifier (most_frequent)
MSE: 142.10344827586206 
0.2689655172413793

Dummy Classifier (uniform)
MSE: 428.82758620689657 
0.034482758620689655

Dummy Classifier (constant)
MSE: 142.10344827586206 
0.2689655172413793

Decision Tree (depth 5)
MSE: 222.46896551724137 
0.19310344827586207

Random Forest (depth 5)
MSE: 144.7103448275862 
0.2689655172413793

Linear SVM
MSE: 161.39310344827587 
0.27586206896551724

RBF SVM
MSE: 142.10344827586206 
0.2689655172413793



In [11]:
Xy_data_set_scaled[0]

array([[-1.02233988, -0.54240738, -0.00859682, ..., -0.00341818,
        -0.45296523, -0.23517129],
       [-0.71751671, -0.50148874, -0.40913128, ..., -0.59775964,
         0.22997398, -0.23517129],
       [ 0.08049625, -0.5407192 , -0.33234964, ...,  0.72461021,
         0.26032683, -0.23517129],
       ...,
       [-0.23293073, -0.0073686 , -0.55359677, ..., -0.68528799,
        -0.45296523, -0.23517129],
       [-0.45183968,  0.07460863, -0.54724711, ..., -0.51382288,
        -0.4377888 , -0.29457974],
       [-0.16065369, -0.29929223, -0.61829382, ..., -0.59563129,
         1.77796952, -0.23517129]])